# **Trend fitting for Horn of Africa flood study**

Use shift fit with GMST and IOD as covariates

## **Additional functions**

In [1]:
source("../../wwa-archive/methods_extRemes.r")

prep_window <- function(rc = c(1,1), w = 4, h = 4, ...) { options(repr.plot.width = rc[2]*w, repr.plot.height = rc[1]*h, repr.plot.res = 200); par(mfrow = rc, pch = 20, ...) }
load_ts <- function(fnm, col.names) { read.csv(fnm, comment.char = "#", sep = " ", header = F, col.names = col.names) }

In [2]:
rp <- function(mdl, ev, qcov) unname(1/pextRemes(mdl, ev, qcov = qcov, lower.tail = F))
rl <- function(mdl, rp, qcov) {
    pars <- findpars(mdl, qcov = qcov)
    unname(rlevd(rp, loc = sum(pars$location), scale = pars$scale, shape = pars$shape))
}
dI <- function(mdl, rp, qc1, qc2, rel = F) if(rel) { (rl(mdl, rp, qc1) - rl(mdl, rp, qc2)) / rl(mdl, rp, qc2) * 100 } else { rl(mdl, rp, qc1) - rl(mdl, rp, qc2) }
pr <- function(mdl, ev, qc1, qc2) rp(mdl, ev, qc2) / rp(mdl, ev, qc1)

In [16]:
mdl_res <- function(mdl) {
        
    pars <- strip(mdl)
    event_2023 <- mdl$x[mdl$cov.data$year == 2023]
    
    qcov_2023 = make.qcov(mdl, vals = list(mu1 = gmst_2023, mu2 = iod_2023))
    qcov_hist = make.qcov(mdl, vals = list(mu1 = gmst_2023 - 1.2, mu2 = iod_2023))
    qcov_neut = make.qcov(mdl, vals = list(mu1 = gmst_2023, mu2 = 0))
    qcov_neuthist = make.qcov(mdl, vals = list(mu1 = gmst_2023 - 1.2, mu2 = 0))

    rp_2023 <- rp(mdl, event_2023, qcov = qcov_2023)
    rp_hist <- rp(mdl, event_2023, qcov = qcov_hist)
    rp_neut <- rp(mdl, event_2023, qcov = qcov_neut)
    rp_neuthist <- rp(mdl, event_2023, qcov = qcov_neuthist)

    # hacky, but better than nothing - will give correct abs/relative value depending on fit type
    if(is.infinite(rp_2023)) rp_2023 <- 100

    rl_hist <- rl(mdl, rp_2023, qcov_hist)
    rl_neut <- rl(mdl, rp_2023, qcov_neut)
    rl_neuthist <- rl(mdl, rp_2023, qcov_neuthist)

    pr_hist <- pr(mdl, event_2023, qcov_2023, qcov_hist)
    pr_neut <- pr(mdl, event_2023, qcov_2023, qcov_neut)
    pr_neuthist <- pr(mdl, event_2023, qcov_2023, qcov_neuthist)

    dI_hist <- dI(mdl, rp_2023, qcov_2023, qcov_hist, rel = F)
    dI_neut <- dI(mdl, rp_2023, qcov_2023, qcov_neut, rel = F)
    dI_neuthist <- dI(mdl, rp_2023, qcov_2023, qcov_neuthist, rel = F)

    rdI_hist <- dI(mdl, rp_2023, qcov_2023, qcov_hist, rel = T)
    rdI_neut <- dI(mdl, rp_2023, qcov_2023, qcov_neut, rel = T)
    rdI_neuthist <- dI(mdl, rp_2023, qcov_2023, qcov_neuthist, rel = T)

    c(pars, sapply(c("event_2023", "rl_hist", "rl_neut", "rl_neuthist", "dI_hist", "dI_neut", "dI_neuthist", "rdI_hist", "rdI_neut", "rdI_neuthist", "rp_2023", "rp_hist", "rp_neut", "rp_neuthist", "pr_hist", "pr_neut", "pr_neuthist"), 
                   get, envir = environment()))
}

In [20]:
boot_res <- function(mdl, nsamp = 1000, seed = 42) {
    
    set.seed(seed)
    mdl_df <- mdl$cov.data
    res <- sapply(1:nsamp, function(i) {
        # print(i)
        lres <- NA
        while(is.na(lres[1])) {
            boot_df <- mdl_df[sample(1:nrow(mdl_df), replace = T),]
            lres <- tryCatch({
                boot_mdl <- suppressWarnings(fevd(unlist(boot_df[varnm]), boot_df, location.fun = ~ gmst + iod))
                mdl_res(boot_mdl)
            }, error = function(cond) {return(NA)})
        }
        lres
    })
    rbind("est" = mdl_res(mdl), apply(res, 1, quantile, c(0.025, 0.975), na.rm = T))
}


## **Properties of IOD for validation**

In [109]:
iod_ts <- iod$iod

set.seed(42)
iod_ci <- sapply(1:1000, function(i) {
    iod_boot <- sample(iod_ts, replace = T)
    c("mean" = mean(iod_boot), "sd" = sd(iod_boot))
})
t(rbind("est" = c("mean" = mean(iod_ts), "sd" = sd(iod_ts)), apply(iod_ci, 1, quantile, c(0.05, 0.95))))

,est,5%,95%
mean,0.002921575,-0.06370119,0.0669238
sd,0.512850929,0.44919538,0.5754951


---
# **Shift fit**

## **Observations**

In [ ]:
cov_lag <- "SON"

gmst <- load_ts("ts/gmst.dat", col.names = c("year", "gmst"))
iod <- load_ts(paste0("ts/iod_", toupper(cov_lag), ".dat"), col.names = c("year", "iod"))
nino <- load_ts(paste0("ts/nino3.4-detrended_", toupper(cov_lag), ".dat"), col.names = c("year", "nino34"))

gmst_2023 <- gmst$gmst[gmst$year == 2023]
iod_2023 <- iod$iod[iod$year == 2023]

In [22]:
dataset <- "tamsat"; varnm <- "rx30day"

# invisible(sapply(c("mswep", "tamsat")[1], function(dataset) {
#     invisible(sapply(c("rx3day", "rx30day")[1], function(varnm) {
                
        ts <- merge(load_ts(paste0("ts/HoA-flooding_", dataset, "_OND_rx3day.dat"), col.names = c("year", "rx3day")),
                    load_ts(paste0("ts/HoA-flooding_", dataset, "_OND_30day_mean.dat"), col.names = c("year", "rx30day")))

        df <- merge(merge(merge(gmst, iod), nino), ts)

        event_2023 <- df[df$year == 2023, varnm]
        
        mdl_shift <- fevd(unlist(df[varnm]), df, location.fun = ~ gmst + iod)

        res <- boot_res(mdl_shift)
        
        write.csv(res, paste0("res/res-obs_shift-fit_", varnm, "_", dataset, "_", cov_lag, ".csv"))
#     }))
# }))

In [76]:
# compile results into a single file
incl <- c("scale", "shape", "event_2023", "rp_2023", "pr_hist", "pr_neut", "dI_hist", "dI_neut", "rdI_hist", "rdI_neut")
res_compiled <- t(sapply(list.files("res", pattern = "res-obs_shift-fit_", full.names = T), function(fnm) setNames(unlist(c(read.csv(fnm)[,incl])), paste0(rep(incl, each = 3), c("_est", "_lower", "_upper")))))
rownames(res_compiled) <- sapply(rownames(res_compiled), function(s) paste0(strsplit(s, "_")[[1]][3:4], collapse = "_"))
write.csv(res_compiled, "res/res-obs_shift-fit.csv")

,est,5%,95%
mean,0.002921575,-0.06370119,0.0669238
sd,0.512850929,0.44919538,0.5754951
